In [0]:
%%sh

aws s3 ls s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-05-30/
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date=2020-05-30/


In [0]:

print spark.read.option("basePath","s3://b2c-prod-data-pipeline-qa/aa.store").parquet( "s3://b2c-prod-data-pipeline-qa/aa.store/result_store_data_v1_20*-[2,3,4,5,6,7,8,9,11,12]*/").filter("raw_store_id  not in (2, 1002)").select("date").orderBy("date", ascending=False).distinct().show()

print spark.read.option("basePath", "s3://b2c-prod-data-pipeline-qa/aa.store/").parquet("s3://b2c-prod-data-pipeline-qa/aa.store/result_store_data_v1_2015-01-02").filter("raw_store_id not in (2, 1002) ").select("date").distinct().orderBy("date", ascending=False).show(500)


In [0]:

start = "2016-09-03"
end = "2016-09-30"
# end = "2012-05-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
sar_list=list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))
    if (real_date1 + datetime.timedelta(days)).weekday() == 5:
        temp=list()
        while dates:
            temp.append(dates.pop())
        sar_list.append({real_date1 + datetime.timedelta(days):temp})

print sar_list

In [0]:


from pyspark.sql import types as T
from pyspark.sql import functions as F
from aadatapipelinecore.core.utils.spark import eject_all_caches
from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline import type_
from applications.common.parser import SqlParser
from applications.common.executor import SqlExecutor
from applications.auto_pipeline.transform import _view
import datetime


spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy

class DryRunSqlExecutor(SqlExecutor):
    def _verify_tasks(self):
        pass


def run(spark, raw_data, sql_text, dry_run=True):
    urn = Urn(namespace=raw_data["namespace"])
    source_data_list = raw_data.pop("source")
    raw_data.update(raw_data.pop("options"))
    _view(spark, sql_text, None, source_data_list)
    context = raw_data
    tasks = SqlParser(spark, sql_text, context).parse()
    if dry_run:
        sql_executor = DryRunSqlExecutor
    else:
        sql_executor = SqlExecutor
    sql_executor(urn, spark, tasks, type_.EventType.TRANSFORM, context).run()





sql_text = """
-- compare raw vs unified data
WITH compared_data_rank AS (
    SELECT * from store_unified where 
);



"""


start = '2020-03-01'
end = '2020-03-01'
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))

# dates=['2020-03-01','2020-01-24',"2019-12-06","2020-01-02","2020-01-05","2020-01-10","2020-01-11","2020-01-13","2020-01-20"]

country_code_df = d1.union(d2).where("country_code is not null").cache()
country_code_df = country_code_df.withColumnRenamed("store_id", "country_code_store_id")
print 'country mapping table'
country_code_df.show(2)
country_code_df.createOrReplaceTempView("country_code_mapping")


namespace = "aa.store.market-size.v1"
for test_date in dates:

    ingest_msg = {
        "namespace": "aa.store.market-size.v1",
        "job_type": "routine",
        "options": {},
        "source": [
             {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "rank_unified",
                "path": [
                    "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/date={}".format(
                        test_date)],
            }, {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "store_unified",
                "path": [
                    "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}".format(
                        test_date)],
            }
        ]
    }
    run(spark, ingest_msg, sql_text)
    result = spark.sql("select * from miss_data_rank").show()
    # result.write.format("delta").save("s3://b2c-prod-data-pipeline-qa/aa.store/result_rank_data_v2_{}/".format(start),
    #                                   mode="append",
    #                                   partitionBy=["date"])


    result_store = spark.sql("select * from miss_data_store").show()
    # result_store.write.format("delta").save("s3://b2c-prod-data-pipeline-qa/aa.store/result_store_data_v2_{}/".format(start),
    #                                   mode="append",
    #                                   partitionBy=["date"])

    eject_all_caches(spark)



In [0]:
%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
select  app_id,feed_id,store_id,estimate,category_id ,device_id, rank, date from plproxy.execute_select_nestloop(\$proxy\$ 
select  distinct app_id,feed_id,store_id,estimate,category_id ,device_id, rank, date
    from aa.app_store_daily_estimate_0
    where 
      app_id = 20600011872874  and date between '2020-05-10' and '2020-05-10' and store_id=11 order by category_id asc, date desc limit 500 \$proxy\$) tbl 
      (app_id BIGINT,feed_id SMALLINT,store_id INT, estimate INT, category_id INT, device_id SMALLINT, rank INT, date Date ) order by category_id  asc, date desc limit 500 ;

EOF



In [0]:

df_raw_1 = spark.read.parquet("s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-05-10").cache()
df_unified_1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date=2020-05-10").cache()

df_raw_1.filter("store_id=11 and platform='android' and feed=1").show(5)
df_unified_1.filter("country_code='BE' and device_code='android-all' ").show()

In [0]:


from pyspark.sql import types as T
from pyspark.sql import functions as F
from aadatapipelinecore.core.utils.spark import eject_all_caches
from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline import type_
from applications.common.parser import SqlParser
from applications.common.executor import SqlExecutor
from applications.auto_pipeline.transform import _view
import datetime


spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy

class DryRunSqlExecutor(SqlExecutor):
    def _verify_tasks(self):
        pass


def run(spark, raw_data, sql_text, dry_run=True):
    urn = Urn(namespace=raw_data["namespace"])
    source_data_list = raw_data.pop("source")
    raw_data.update(raw_data.pop("options"))
    _view(spark, sql_text, None, source_data_list)
    context = raw_data
    tasks = SqlParser(spark, sql_text, context).parse()
    if dry_run:
        sql_executor = DryRunSqlExecutor
    else:
        sql_executor = SqlExecutor
    sql_executor(urn, spark, tasks, type_.EventType.TRANSFORM, context).run()


sql_text = """

-- mapping feed as metrc in raw
WITH feed_metric AS (
select *, 'free_app_download' as metric, "ios-phone" as device_code from rank_raw where  feed='0' and platform='ios'
UNION ALL
select *, 'paid_app_download' as metric, "ios-phone" as device_code from rank_raw where  feed='1' and platform='ios'
UNION ALL
select *, 'revenue' as metric , "ios-phone" as device_code from rank_raw where  feed='2' and platform='ios'
UNION ALL
select *, 'free_app_download' as metric, "ios-tablet" as device_code from rank_raw where  feed='101' and platform='ios'
UNION ALL
select *, 'paid_app_download' as metric, "ios-tablet" as device_code from rank_raw  where  feed='100' and platform='ios'
UNION ALL
select *, 'revenue' as metric, "ios-tablet" as device_code from rank_raw  where  feed='102' and platform='ios'
UNION ALL
select *, 'free_app_download' as metric, "ios-all" as device_code from rank_raw  where  feed='1000' and platform='ios'
UNION ALL
select *, 'paid_app_download' as metric, "ios-all" as device_code from rank_raw  where  feed='1001' and platform='ios'
UNION ALL
select *, 'revenue' as metric, "ios-all" as device_code from rank_raw  where  feed='1002' and platform='ios'
UNION ALL
select *, 'free_app_download' as metric , "android-all" as device_code from rank_raw   where  feed='0' and platform='android'
UNION ALL
select *, 'paid_app_download' as metric, "android-all" as device_code from rank_raw  where  feed='1' and platform='android'
UNION ALL
select *, 'revenue' as metric,  "android-all" as device_code from rank_raw  where  feed='2' and platform='android'
);


-- select tested column from raw data
WITH metric_raw_data AS (
SELECT id, category_id as raw_category_id,rank,store_id as raw_store_id , metric,device_code,date , platform from feed_metric where store_id not in (3,4,5,6, 1002,1003, 1005,1004, 1006,1007)
);


-- group by and count data in raw data
WITH group_by_raw AS (
SELECT count(id) AS total_count , raw_category_id, raw_store_id, metric,device_code,date,platform from metric_raw_data where raw_store_id not in (3,4,5,6, 1002,1003, 1004, 1005, 1006,1007) group by raw_category_id, raw_store_id, metric,device_code,date, platform
);


-- pivot metric column
WITH pivot_metric_rank_raw AS (

SELECT 
free_app_download,revenue, paid_app_download, raw_category_id,raw_store_id,device_code, platform,date
FROM
      group_by_raw
 PIVOT (
    max(total_count) 
	FOR metric IN ('free_app_download','revenue', 'paid_app_download')
  )
);




-- select tested column from raw data
WITH metric_raw_store_data AS (
SELECT distinct id, est, store_id as raw_store_id , metric,device_code, date , platform from feed_metric where store_id not in (3,4,5,6, 1002,1003, 1005,1004, 1006,1007)

);


-- group by and count data in raw data
WITH group_by_store_raw AS (
SELECT count(est) AS total_count ,raw_store_id, metric,device_code,date,platform from metric_raw_store_data where raw_store_id not in (3,4,5,6,1002,1003, 1004, 1005, 1006,1007) group by raw_store_id, metric,device_code,date, platform
);


-- pivot metric column
WITH pivot_metric_store_raw AS (

SELECT 
free_app_download,revenue, paid_app_download,raw_store_id,device_code, platform, date
FROM
      group_by_store_raw
 PIVOT (
    max(total_count) 
	FOR metric IN ('free_app_download','revenue', 'paid_app_download')
  )
);



-- map raw with category
WITH category_mapping_raw AS (

SELECT * from ( select *, 'ios' as mapping_platform from category_mapping_deminsion_service where market_code='apple-store' 
UNION ALL select *, 'android' as mapping_platform from category_mapping_deminsion_service where market_code='google-play'
 ) as mapping right join pivot_metric_rank_raw on mapping.legacy_category_id=pivot_metric_rank_raw.raw_category_id and 
mapping.mapping_platform=pivot_metric_rank_raw.platform
);


-- map raw with rank country_code
WITH country_category_mapping_rank_raw AS (
select date, raw_store_id, country_code,device_code,category_id,free_app_download,paid_app_download,revenue from country_code_mapping right join category_mapping_raw on country_code_mapping.country_code_store_id=category_mapping_raw.raw_store_id and country_code_mapping.market_code=category_mapping_raw.platform
);



-- map raw with store country_code
WITH country_mapping_store_raw AS (
select date, raw_store_id, country_code,device_code,free_app_download,paid_app_download,revenue from country_code_mapping right join pivot_metric_store_raw on country_code_mapping.country_code_store_id=pivot_metric_store_raw.raw_store_id and country_code_mapping.market_code=pivot_metric_store_raw.platform
);


-- group by unified data
WITH unified_group_data AS (
select count(app_id) as unified_count_app_id, count(free_app_download) as unified_count_free_app_download, count(paid_app_download) as unified_count_paid_app_download, count(revenue) as unified_count_revenue,
  country_code as unified_country_code, device_code as unified_device_code, category_id as unified_category_id from ( select distinct  app_id, free_app_download, paid_app_download, revenue, country_code, device_code, category_id from  rank_unified ) as unified
group by  category_id,  country_code,  device_code );


-- group by unified data
WITH unified_group_data_store AS (
select count(app_id) as unified_count_app_id, count(free_app_download) as unified_count_free_app_download, count(paid_app_download) as unified_count_paid_app_download, count(revenue) as unified_count_revenue,
  country_code as unified_country_code, device_code as unified_device_code from ( select distinct  app_id, free_app_download, paid_app_download, revenue, country_code, device_code from  store_unified ) as unified
group by   country_code,  device_code );




-- compare raw vs unified data
WITH compared_data_rank AS (
    SELECT * from country_category_mapping_rank_raw left join unified_group_data on unified_group_data.unified_country_code==country_category_mapping_rank_raw.country_code and unified_group_data.unified_category_id==country_category_mapping_rank_raw.category_id and unified_group_data.unified_device_code==country_category_mapping_rank_raw.device_code
);

WITH miss_data_rank AS (
select * from compared_data_rank where unified_count_paid_app_download!=paid_app_download or unified_count_free_app_download != free_app_download  or unified_count_revenue != revenue or unified_count_app_id is null
)



-- compare raw vs unified data store
WITH compared_store_data AS (
    SELECT * from country_mapping_store_raw left join unified_group_data_store on unified_group_data_store.unified_country_code==country_mapping_store_raw.country_code and unified_group_data_store.unified_device_code==country_mapping_store_raw.device_code
);


WITH miss_data_store AS (
select * from compared_store_data where free_app_download!=unified_count_free_app_download or paid_app_download!=unified_count_paid_app_download or revenue!=unified_count_revenue or unified_count_app_id is null
)


"""


start = '2020-05-10'
end = '2020-05-11'
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))

# dates=['2020-03-01','2020-01-24',"2019-12-06","2020-01-02","2020-01-05","2020-01-10","2020-01-11","2020-01-13","2020-01-20"]

d1 = spark.read.csv("s3://b2c-prod-dca-store-estimates/store_back/dimension/IOS_COUNTRY_MAPPING",
                    sep="\t").withColumnRenamed("_c0", "store_id").withColumnRenamed("_c1",
                                                                                     "country_code").withColumn(
    "market_code", F.lit("ios"))
d1 = spark.createDataFrame([(0, 'WW', 'Worldwide', 'ios')],
                           schema=["store_id", "country_code", "_c2", "market_code"]).union(d1)

d2 = spark.read.csv("s3://b2c-prod-dca-store-estimates/store_back/dimension/ANDROID_COUNTRY_MAPPING",
                    sep="\t").withColumnRenamed("_c0", "store_id").withColumnRenamed("_c1",
                                                                                     "country_code").withColumn(
    "market_code", F.lit("android"))
country_code_df = d1.union(d2).where("country_code is not null").cache()
country_code_df = country_code_df.withColumnRenamed("store_id", "country_code_store_id")
print 'country mapping table'
country_code_df.show(2)
country_code_df.createOrReplaceTempView("country_code_mapping")

category_mapping_table = spark.read.parquet(
    "s3://b2c-prod-data-pipeline-qa/aa.store/store_cateogry_mapping")
category_mapping_table.createOrReplaceTempView("category_mapping_deminsion_service")

namespace = "aa.store.market-size.v1"
for test_date in dates:
    print test_date
    ingest_msg = {
        "namespace": "aa.store.market-size.v1",
        "job_type": "routine",
        "options": {},
        "source": [
            {
                "name": "rank_raw",
                "data_encoding": "parquet",
                "compression": "gzip",
                "path": [
                    "s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date={}/".format(
                        test_date)],
            }, {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "rank_unified",
                "path": [
                    "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/date={}".format(
                        test_date)],
            }, {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "store_unified",
                "path": [
                    "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}".format(
                        test_date)],
            }
        ]
    }
    run(spark, ingest_msg, sql_text)
    result = spark.sql("select * from miss_data_rank")
    result.show()
    # result.write.format("delta").save("s3://b2c-prod-data-pipeline-qa/aa.store/result_rank_data_v2_{}/".format(start),
    #                                   mode="append",
    #                                   partitionBy=["date"])


    result_store = spark.sql("select * from miss_data_store")
    result_store.show()
    # result_store.write.format("delta").save("s3://b2c-prod-data-pipeline-qa/aa.store/result_store_data_v2_{}/".format(start),
    #                                   mode="append",
    #                                   partitionBy=["date"])

    # eject_all_caches(spark)



In [0]:


from pyspark.sql import types as T
from pyspark.sql import functions as F
from aadatapipelinecore.core.utils.spark import eject_all_caches
from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline import type_
from applications.common.parser import SqlParser
from applications.common.executor import SqlExecutor
from applications.auto_pipeline.transform import _view
import datetime


spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy

class DryRunSqlExecutor(SqlExecutor):
    def _verify_tasks(self):
        pass


def run(spark, raw_data, sql_text, dry_run=True):
    urn = Urn(namespace=raw_data["namespace"])
    source_data_list = raw_data.pop("source")
    raw_data.update(raw_data.pop("options"))
    _view(spark, sql_text, None, source_data_list)
    context = raw_data
    tasks = SqlParser(spark, sql_text, context).parse()
    if dry_run:
        sql_executor = DryRunSqlExecutor
    else:
        sql_executor = SqlExecutor
    sql_executor(urn, spark, tasks, type_.EventType.TRANSFORM, context).run()


sql_text = """

-- mapping feed as metrc in raw
WITH feed_metric AS (
select *, 'free_app_download' as metric, "ios-phone" as device_code from rank_raw where  feed='0' and platform='ios'
UNION ALL
select *, 'paid_app_download' as metric, "ios-phone" as device_code from rank_raw where  feed='1' and platform='ios'
UNION ALL
select *, 'revenue' as metric , "ios-phone" as device_code from rank_raw where  feed='2' and platform='ios'
UNION ALL
select *, 'free_app_download' as metric, "ios-tablet" as device_code from rank_raw where  feed='101' and platform='ios'
UNION ALL
select *, 'paid_app_download' as metric, "ios-tablet" as device_code from rank_raw  where  feed='100' and platform='ios'
UNION ALL
select *, 'revenue' as metric, "ios-tablet" as device_code from rank_raw  where  feed='102' and platform='ios'
UNION ALL
select *, 'free_app_download' as metric, "ios-all" as device_code from rank_raw  where  feed='1000' and platform='ios'
UNION ALL
select *, 'paid_app_download' as metric, "ios-all" as device_code from rank_raw  where  feed='1001' and platform='ios'
UNION ALL
select *, 'revenue' as metric, "ios-all" as device_code from rank_raw  where  feed='1002' and platform='ios'
UNION ALL
select *, 'free_app_download' as metric , "android-all" as device_code from rank_raw   where  feed='0' and platform='android'
UNION ALL
select *, 'paid_app_download' as metric, "android-all" as device_code from rank_raw  where  feed='1' and platform='android'
UNION ALL
select *, 'revenue' as metric,  "android-all" as device_code from rank_raw  where  feed='2' and platform='android'
);


-- select tested column from raw data
WITH metric_raw_data AS (
SELECT id, category_id as raw_category_id,rank,store_id as raw_store_id , metric,device_code,date , platform from feed_metric where store_id not in (3,4,5,6, 1002,1003, 1005,1004, 1006,1007)
);


-- group by and count data in raw data
WITH group_by_raw AS (
SELECT count(id) AS total_count , raw_category_id, raw_store_id, metric,device_code,date,platform from metric_raw_data where raw_store_id not in (3,4,5,6, 1002,1003, 1004, 1005, 1006,1007) group by raw_category_id, raw_store_id, metric,device_code,date, platform
);


-- pivot metric column
WITH pivot_metric_rank_raw AS (

SELECT 
free_app_download,revenue, paid_app_download, raw_category_id,raw_store_id,device_code, platform,date
FROM
      group_by_raw
 PIVOT (
    max(total_count) 
	FOR metric IN ('free_app_download','revenue', 'paid_app_download')
  )
);




-- select tested column from raw data
WITH metric_raw_store_data AS (
SELECT distinct id, est, store_id as raw_store_id , metric,device_code, date , platform from feed_metric where store_id not in (3,4,5,6, 1002,1003, 1005,1004, 1006,1007)

);


-- group by and count data in raw data
WITH group_by_store_raw AS (
SELECT count(est) AS total_count ,raw_store_id, metric,device_code,date,platform from metric_raw_store_data where raw_store_id not in (3,4,5,6,1002,1003, 1004, 1005, 1006,1007) group by raw_store_id, metric,device_code,date, platform
);


-- pivot metric column
WITH pivot_metric_store_raw AS (

SELECT 
free_app_download,revenue, paid_app_download,raw_store_id,device_code, platform, date
FROM
      group_by_store_raw
 PIVOT (
    max(total_count) 
	FOR metric IN ('free_app_download','revenue', 'paid_app_download')
  )
);



-- map raw with category
WITH category_mapping_raw AS (

SELECT * from ( select *, 'ios' as mapping_platform from category_mapping_deminsion_service where market_code='apple-store' 
UNION ALL select *, 'android' as mapping_platform from category_mapping_deminsion_service where market_code='google-play'
 ) as mapping right join pivot_metric_rank_raw on mapping.legacy_category_id=pivot_metric_rank_raw.raw_category_id and 
mapping.mapping_platform=pivot_metric_rank_raw.platform
);


-- map raw with rank country_code
WITH country_category_mapping_rank_raw AS (
select date, raw_store_id, country_code,device_code,category_id,free_app_download,paid_app_download,revenue from country_code_mapping right join category_mapping_raw on country_code_mapping.country_code_store_id=category_mapping_raw.raw_store_id and country_code_mapping.market_code=category_mapping_raw.platform
);



-- map raw with store country_code
WITH country_mapping_store_raw AS (
select date, raw_store_id, country_code,device_code,free_app_download,paid_app_download,revenue from country_code_mapping right join pivot_metric_store_raw on country_code_mapping.country_code_store_id=pivot_metric_store_raw.raw_store_id and country_code_mapping.market_code=pivot_metric_store_raw.platform
);


-- group by unified data
WITH unified_group_data AS (
select count(app_id) as unified_count_app_id, count(free_app_download) as unified_count_free_app_download, count(paid_app_download) as unified_count_paid_app_download, count(revenue) as unified_count_revenue,
  country_code as unified_country_code, device_code as unified_device_code, category_id as unified_category_id from ( select distinct  app_id, free_app_download, paid_app_download, revenue, country_code, device_code, category_id from  rank_unified ) as unified
group by  category_id,  country_code,  device_code );


-- group by unified data
WITH unified_group_data_store AS (
select count(app_id) as unified_count_app_id, count(free_app_download) as unified_count_free_app_download, count(paid_app_download) as unified_count_paid_app_download, count(revenue) as unified_count_revenue,
  country_code as unified_country_code, device_code as unified_device_code from ( select distinct  app_id, free_app_download, paid_app_download, revenue, country_code, device_code from  store_unified ) as unified
group by   country_code,  device_code );




-- compare raw vs unified data
WITH compared_data_rank AS (
    SELECT * from country_category_mapping_rank_raw left join unified_group_data on unified_group_data.unified_country_code==country_category_mapping_rank_raw.country_code and unified_group_data.unified_category_id==country_category_mapping_rank_raw.category_id and unified_group_data.unified_device_code==country_category_mapping_rank_raw.device_code
);

WITH miss_data_rank AS (
select * from compared_data_rank where unified_count_paid_app_download!=paid_app_download or unified_count_free_app_download != free_app_download  or unified_count_revenue != revenue or unified_count_app_id is null
)



-- compare raw vs unified data store
WITH compared_store_data AS (
    SELECT * from country_mapping_store_raw left join unified_group_data_store on unified_group_data_store.unified_country_code==country_mapping_store_raw.country_code and unified_group_data_store.unified_device_code==country_mapping_store_raw.device_code
);


WITH miss_data_store AS (
select * from compared_store_data where free_app_download!=unified_count_free_app_download or paid_app_download!=unified_count_paid_app_download or revenue!=unified_count_revenue or unified_count_app_id is null
)


"""


start = '2020-04-07'
end = '2020-05-28'
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))

# dates=['2020-03-01','2020-01-24',"2019-12-06","2020-01-02","2020-01-05","2020-01-10","2020-01-11","2020-01-13","2020-01-20"]

d1 = spark.read.csv("s3://b2c-prod-dca-store-estimates/store_back/dimension/IOS_COUNTRY_MAPPING",
                    sep="\t").withColumnRenamed("_c0", "store_id").withColumnRenamed("_c1",
                                                                                     "country_code").withColumn(
    "market_code", F.lit("ios"))
d1 = spark.createDataFrame([(0, 'WW', 'Worldwide', 'ios')],
                           schema=["store_id", "country_code", "_c2", "market_code"]).union(d1)

d2 = spark.read.csv("s3://b2c-prod-dca-store-estimates/store_back/dimension/ANDROID_COUNTRY_MAPPING",
                    sep="\t").withColumnRenamed("_c0", "store_id").withColumnRenamed("_c1",
                                                                                     "country_code").withColumn(
    "market_code", F.lit("android"))
country_code_df = d1.union(d2).where("country_code is not null").cache()
country_code_df = country_code_df.withColumnRenamed("store_id", "country_code_store_id")
print 'country mapping table'
country_code_df.show(2)
country_code_df.createOrReplaceTempView("country_code_mapping")

category_mapping_table = spark.read.parquet(
    "s3://b2c-prod-data-pipeline-qa/aa.store/store_cateogry_mapping")
category_mapping_table.createOrReplaceTempView("category_mapping_deminsion_service")

namespace = "aa.store.market-size.v1"
for test_date in dates:
    print test_date
    ingest_msg = {
        "namespace": "aa.store.market-size.v1",
        "job_type": "routine",
        "options": {},
        "source": [
            {
                "name": "rank_raw",
                "data_encoding": "parquet",
                "compression": "gzip",
                "path": [
                    "s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date={}/".format(
                        test_date)],
            }, {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "rank_unified",
                "path": [
                    "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/date={}".format(
                        test_date)],
            }, {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "store_unified",
                "path": [
                    "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}".format(
                        test_date)],
            }
        ]
    }
    run(spark, ingest_msg, sql_text)
    result = spark.sql("select * from miss_data_rank")
    result.show()
    result.write.format("delta").save("s3://b2c-prod-data-pipeline-qa/aa.store/result_rank_data_v2_{}/".format(start),
                                      mode="append",
                                      partitionBy=["date"])


    result_store = spark.sql("select * from miss_data_store")
    result_store.show()
    result_store.write.format("delta").save("s3://b2c-prod-data-pipeline-qa/aa.store/result_store_data_v2_{}/".format(start),
                                      mode="append",
                                      partitionBy=["date"])

    eject_all_caches(spark)



In [0]:



from pyspark.sql import types as T
from pyspark.sql import functions as F

from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline import type_
from applications.common.parser import SqlParser
from applications.common.executor import SqlExecutor
from applications.auto_pipeline.transform import _view
import datetime
from aadatapipelinecore.core.utils.spark import eject_all_caches


spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy

class DryRunSqlExecutor(SqlExecutor):
    def _verify_tasks(self):
        pass


def run(spark, raw_data, sql_text, dry_run=True):
    urn = Urn(namespace=raw_data["namespace"])
    source_data_list = raw_data.pop("source")
    raw_data.update(raw_data.pop("options"))
    _view(spark, sql_text, None, source_data_list)
    context = raw_data
    tasks = SqlParser(spark, sql_text, context).parse()
    if dry_run:
        sql_executor = DryRunSqlExecutor
    else:
        sql_executor = SqlExecutor
    sql_executor(urn, spark, tasks, type_.EventType.TRANSFORM, context).run()


sql_text = """

-- mapping feed as metrc in raw
WITH feed_metric AS (
select *, 'free_app_download' as metric, "ios-phone" as device_code from rank_raw where  feed='0' and platform='ios'
UNION ALL
select *, 'paid_app_download' as metric, "ios-phone" as device_code from rank_raw where  feed='1' and platform='ios'
UNION ALL
select *, 'revenue' as metric , "ios-phone" as device_code from rank_raw where  feed='2' and platform='ios'
UNION ALL
select *, 'free_app_download' as metric, "ios-tablet" as device_code from rank_raw where  feed='101' and platform='ios'
UNION ALL
select *, 'paid_app_download' as metric, "ios-tablet" as device_code from rank_raw  where  feed='100' and platform='ios'
UNION ALL
select *, 'revenue' as metric, "ios-tablet" as device_code from rank_raw  where  feed='102' and platform='ios'
UNION ALL
select *, 'free_app_download' as metric, "ios-all" as device_code from rank_raw  where  feed='1000' and platform='ios'
UNION ALL
select *, 'paid_app_download' as metric, "ios-all" as device_code from rank_raw  where  feed='1001' and platform='ios'
UNION ALL
select *, 'revenue' as metric, "ios-all" as device_code from rank_raw  where  feed='1002' and platform='ios'
UNION ALL
select *, 'free_app_download' as metric , "android-all" as device_code from rank_raw   where  feed='0' and platform='android'
UNION ALL
select *, 'paid_app_download' as metric, "android-all" as device_code from rank_raw  where  feed='1' and platform='android'
UNION ALL
select *, 'revenue' as metric,  "android-all" as device_code from rank_raw  where  feed='2' and platform='android'
);


-- select tested column from raw data
WITH metric_raw_data AS (
SELECT id, category_id as raw_category_id,rank,store_id as raw_store_id , metric,device_code,date , platform from feed_metric where store_id not in (2,3,4,5,6,1002, 1003, 1005,1004, 1006,1007)
);


-- group by and count data in raw data
WITH group_by_raw AS (
SELECT count(id) AS total_count , raw_category_id, raw_store_id, metric,device_code,date,platform from metric_raw_data where raw_store_id not in (2,3,4,5,6, 1002, 1003, 1004, 1005, 1006,1007) group by raw_category_id, raw_store_id, metric,device_code,date, platform
);


-- pivot metric column
WITH pivot_metric_rank_raw AS (

SELECT 
free_app_download,revenue, paid_app_download, raw_category_id,raw_store_id,device_code, platform,date
FROM
      group_by_raw
 PIVOT (
    max(total_count) 
	FOR metric IN ('free_app_download','revenue', 'paid_app_download')
  )
);




-- select tested column from raw data
WITH metric_raw_store_data AS (
SELECT distinct id, est, store_id as raw_store_id , metric,device_code,date , platform from feed_metric where store_id not in (2, 3,4,5,6, 1002,1003, 1005,1004, 1006,1007)
);


-- group by and count data in raw data
WITH group_by_store_raw AS (
SELECT count(est) AS total_count ,raw_store_id, metric,device_code,date,platform from metric_raw_store_data where raw_store_id not in (2, 3,4,5,6, 1002, 1003, 1005, 1004, 1006,1007) group by raw_store_id, metric,device_code,date, platform
);


-- pivot metric column
WITH pivot_metric_store_raw AS (

SELECT 
free_app_download,revenue, paid_app_download,raw_store_id,device_code, platform, date
FROM
      group_by_store_raw
 PIVOT (
    max(total_count) 
	FOR metric IN ('free_app_download','revenue', 'paid_app_download')
  )
);



-- map raw with category
WITH category_mapping_raw AS (

SELECT * from ( select *, 'ios' as mapping_platform from category_mapping_deminsion_service where market_code='apple-store' 
UNION ALL select *, 'android' as mapping_platform from category_mapping_deminsion_service where market_code='google-play'
 ) as mapping right join pivot_metric_rank_raw on mapping.legacy_category_id=pivot_metric_rank_raw.raw_category_id and 
mapping.mapping_platform=pivot_metric_rank_raw.platform
);


-- map raw with rank country_code
WITH country_category_mapping_rank_raw AS (
select date, raw_store_id, country_code,device_code,category_id,free_app_download,paid_app_download,revenue from country_code_mapping right join category_mapping_raw on country_code_mapping.country_code_store_id=category_mapping_raw.raw_store_id and country_code_mapping.market_code=category_mapping_raw.platform
);



-- map raw with store country_code
WITH country_mapping_store_raw AS (
select date, raw_store_id, country_code,device_code,free_app_download,paid_app_download,revenue from country_code_mapping right join pivot_metric_store_raw on country_code_mapping.country_code_store_id=pivot_metric_store_raw.raw_store_id and country_code_mapping.market_code=pivot_metric_store_raw.platform
);


-- group by unified data
WITH unified_group_data AS (
select count(app_id) as unified_count_app_id, count(free_app_download) as unified_count_free_app_download, count(paid_app_download) as unified_count_paid_app_download, count(revenue) as unified_count_revenue,
  country_code as unified_country_code, device_code as unified_device_code, category_id as unified_category_id from ( select distinct  app_id, free_app_download, paid_app_download, revenue, country_code, device_code, category_id from  rank_unified ) as unified
group by  category_id,  country_code,  device_code );


-- group by unified data
WITH unified_group_data_store AS (
select count(app_id) as unified_count_app_id, count(free_app_download) as unified_count_free_app_download, count(paid_app_download) as unified_count_paid_app_download, count(revenue) as unified_count_revenue,
  country_code as unified_country_code, device_code as unified_device_code from ( select distinct  app_id, free_app_download, paid_app_download, revenue, country_code, device_code from  store_unified ) as unified
group by   country_code,  device_code );




-- compare raw vs unified data
WITH compared_data_rank AS (
    SELECT * from country_category_mapping_rank_raw left join unified_group_data on unified_group_data.unified_country_code==country_category_mapping_rank_raw.country_code and unified_group_data.unified_category_id==country_category_mapping_rank_raw.category_id and unified_group_data.unified_device_code==country_category_mapping_rank_raw.device_code
);

WITH miss_data_rank AS (
select * from compared_data_rank where unified_count_paid_app_download!=paid_app_download or unified_count_free_app_download != free_app_download  or unified_count_revenue != revenue or unified_count_app_id is null
)



-- compare raw vs unified data store
WITH compared_store_data AS (
    SELECT * from country_mapping_store_raw left join unified_group_data_store on unified_group_data_store.unified_country_code==country_mapping_store_raw.country_code and unified_group_data_store.unified_device_code==country_mapping_store_raw.device_code
);


WITH miss_data_store AS (
select * from compared_store_data where free_app_download!=unified_count_free_app_download or paid_app_download!=unified_count_paid_app_download or revenue!=unified_count_revenue or unified_count_app_id is null
)


"""



start = "2015-01-01"
end = "2015-01-11"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))

# dates=["2013-06-03"]

d1 = spark.read.csv("s3://b2c-prod-dca-store-estimates/store_back/dimension/IOS_COUNTRY_MAPPING",
                    sep="\t").withColumnRenamed("_c0", "store_id").withColumnRenamed("_c1",
                                                                                     "country_code").withColumn(
    "market_code", F.lit("ios"))
d1 = spark.createDataFrame([(0, 'WW', 'Worldwide', 'ios')],
                           schema=["store_id", "country_code", "_c2", "market_code"]).union(d1)

d2 = spark.read.csv("s3://b2c-prod-dca-store-estimates/store_back/dimension/ANDROID_COUNTRY_MAPPING",
                    sep="\t").withColumnRenamed("_c0", "store_id").withColumnRenamed("_c1",
                                                                                     "country_code").withColumn(
    "market_code", F.lit("android"))

country_code_df = d1.union(d2).where("country_code is not null").cache()
country_code_df = country_code_df.withColumnRenamed("store_id", "country_code_store_id")
print 'country mapping table'
country_code_df.show(2)
country_code_df.createOrReplaceTempView("country_code_mapping")

category_mapping_table = spark.read.parquet(
    "s3://b2c-prod-data-pipeline-qa/aa.store/store_cateogry_mapping")
category_mapping_table.createOrReplaceTempView("category_mapping_deminsion_service")

namespace = "aa.store.market-size.v1"
for test_date in dates:
    print test_date
    csv_schema = T.StructType(
        [
            T.StructField("store_id", T.IntegerType(), True),
            T.StructField("date", T.StringType(), True),
            T.StructField("platform_id", T.IntegerType(), True),
            T.StructField("vertical", T.IntegerType(), True),
            T.StructField("feed", T.IntegerType(), True),
            T.StructField("id", T.LongType(), True),
            T.StructField("est", T.IntegerType(), True),
            T.StructField("category_id", T.IntegerType(), True),
            T.StructField("rank", T.IntegerType(), True)
        ]
    )

    raw1 = spark.read.option("basePath",
                             "s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/").schema(
        csv_schema).csv(
        "s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/{}/android/sbe_est_app/*/".format(
            test_date), sep="\t").withColumn("platform", F.lit("android")).cache()
    raw2 = spark.read.option("basePath",
                             "s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/").schema(
        csv_schema).csv(
        "s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/{}/ios/sbe_est_app/*/".format(test_date),
        sep="\t").withColumn("platform", F.lit("ios")).cache()

    df_raw = raw1.union(raw2).cache()
    df_raw.createOrReplaceTempView("rank_raw")

    ingest_msg = {
        "namespace": "aa.store.market-size.v1",
        "job_type": "routine",
        "options": {},
        "source": [
            {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "rank_unified",
                "path": [
                    "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/date={}".format(
                        test_date)],
            }, {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "store_unified",
                "path": [
                    "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}".format(
                        test_date)],
            }
        ]
    }
    run(spark, ingest_msg, sql_text)
    result = spark.sql("select * from miss_data_rank")
    result.show()
    # result.write.format("delta").save("s3://b2c-prod-data-pipeline-qa/aa.store/result_rank_data_v1_{}/".format(test_date),
    #                                   mode="append",
    #                                   partitionBy=["date"])


    result_store = spark.sql("select * from miss_data_store")
    result.show()

    # result_store.write.format("delta").save("s3://b2c-prod-data-pipeline-qa/aa.store/result_store_data_v1_{}/".format(test_date),
    #                                   mode="append",
    #                                   partitionBy=["date"])
    eject_all_caches(spark)


In [0]:

from pyspark.sql import types as T
from pyspark.sql import functions as F

from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline import type_
from applications.common.parser import SqlParser
from applications.common.executor import SqlExecutor
from applications.auto_pipeline.transform import _view
import datetime
from aadatapipelinecore.core.utils.spark import eject_all_caches


spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy

class DryRunSqlExecutor(SqlExecutor):
    def _verify_tasks(self):
        pass


def run(spark, raw_data, sql_text, dry_run=True):
    urn = Urn(namespace=raw_data["namespace"])
    source_data_list = raw_data.pop("source")
    raw_data.update(raw_data.pop("options"))
    _view(spark, sql_text, None, source_data_list)
    context = raw_data
    tasks = SqlParser(spark, sql_text, context).parse()
    if dry_run:
        sql_executor = DryRunSqlExecutor
    else:
        sql_executor = SqlExecutor
    sql_executor(urn, spark, tasks, type_.EventType.TRANSFORM, context).run()


sql_text = """

-- mapping feed as metrc in raw
WITH feed_metric AS (
select *, 'free_app_download' as metric, "ios-phone" as device_code from rank_raw where  feed='0' and platform='ios'
UNION ALL
select *, 'paid_app_download' as metric, "ios-phone" as device_code from rank_raw where  feed='1' and platform='ios'
UNION ALL
select *, 'revenue' as metric , "ios-phone" as device_code from rank_raw where  feed='2' and platform='ios'
UNION ALL
select *, 'free_app_download' as metric, "ios-tablet" as device_code from rank_raw where  feed='101' and platform='ios'
UNION ALL
select *, 'paid_app_download' as metric, "ios-tablet" as device_code from rank_raw  where  feed='100' and platform='ios'
UNION ALL
select *, 'revenue' as metric, "ios-tablet" as device_code from rank_raw  where  feed='102' and platform='ios'
UNION ALL
select *, 'free_app_download' as metric, "ios-all" as device_code from rank_raw  where  feed='1000' and platform='ios'
UNION ALL
select *, 'paid_app_download' as metric, "ios-all" as device_code from rank_raw  where  feed='1001' and platform='ios'
UNION ALL
select *, 'revenue' as metric, "ios-all" as device_code from rank_raw  where  feed='1002' and platform='ios'
UNION ALL
select *, 'free_app_download' as metric , "android-all" as device_code from rank_raw   where  feed='0' and platform='android'
UNION ALL
select *, 'paid_app_download' as metric, "android-all" as device_code from rank_raw  where  feed='1' and platform='android'
UNION ALL
select *, 'revenue' as metric,  "android-all" as device_code from rank_raw  where  feed='2' and platform='android'
);


-- select tested column from raw data
WITH metric_raw_data AS (
SELECT id, category_id as raw_category_id,rank,store_id as raw_store_id , metric,device_code,date , platform from feed_metric where store_id not in (2, 3,4,5,6,1003, 1002,1005,1004, 1006,1007)
);


-- group by and count data in raw data
WITH group_by_raw AS (
SELECT count(id) AS total_count , raw_category_id, raw_store_id, metric,device_code,date,platform from metric_raw_data where raw_store_id not in (2, 3,4,5,6, 1002, 1003, 1004, 1005, 1006,1007) group by raw_category_id, raw_store_id, metric,device_code,date, platform
);


-- pivot metric column
WITH pivot_metric_rank_raw AS (

SELECT 
free_app_download,revenue, paid_app_download, raw_category_id,raw_store_id,device_code, platform,date
FROM
      group_by_raw
 PIVOT (
    max(total_count) 
	FOR metric IN ('free_app_download','revenue', 'paid_app_download')
  )
);




-- select tested column from raw data
WITH metric_raw_store_data AS (
SELECT distinct id, est, store_id as raw_store_id , metric,device_code,date , platform from feed_metric where store_id not in (2,3,4,5,6, 1002, 1003, 1005,1004, 1006,1007)
);


-- group by and count data in raw data
WITH group_by_store_raw AS (
SELECT count(est) AS total_count ,raw_store_id, metric,device_code,date,platform from metric_raw_store_data where raw_store_id not in (2, 3,4,5,6, 1002, 1003, 1005, 1004, 1006,1007) group by raw_store_id, metric,device_code,date, platform
);


-- pivot metric column
WITH pivot_metric_store_raw AS (

SELECT 
free_app_download,revenue, paid_app_download,raw_store_id,device_code, platform, date
FROM
      group_by_store_raw
 PIVOT (
    max(total_count) 
	FOR metric IN ('free_app_download','revenue', 'paid_app_download')
  )
);



-- map raw with category
WITH category_mapping_raw AS (

SELECT * from ( select *, 'ios' as mapping_platform from category_mapping_deminsion_service where market_code='apple-store' 
UNION ALL select *, 'android' as mapping_platform from category_mapping_deminsion_service where market_code='google-play'
 ) as mapping right join pivot_metric_rank_raw on mapping.legacy_category_id=pivot_metric_rank_raw.raw_category_id and 
mapping.mapping_platform=pivot_metric_rank_raw.platform
);


-- map raw with rank country_code
WITH country_category_mapping_rank_raw AS (
select date, raw_store_id, country_code,device_code,category_id,free_app_download,paid_app_download,revenue from country_code_mapping right join category_mapping_raw on country_code_mapping.country_code_store_id=category_mapping_raw.raw_store_id and country_code_mapping.market_code=category_mapping_raw.platform
);



-- map raw with store country_code
WITH country_mapping_store_raw AS (
select date, raw_store_id, country_code,device_code,free_app_download,paid_app_download,revenue from country_code_mapping right join pivot_metric_store_raw on country_code_mapping.country_code_store_id=pivot_metric_store_raw.raw_store_id and country_code_mapping.market_code=pivot_metric_store_raw.platform
);


-- group by unified data
WITH unified_group_data AS (
select count(app_id) as unified_count_app_id, count(free_app_download) as unified_count_free_app_download, count(paid_app_download) as unified_count_paid_app_download, count(revenue) as unified_count_revenue,
  country_code as unified_country_code, device_code as unified_device_code, category_id as unified_category_id from ( select distinct  app_id, free_app_download, paid_app_download, revenue, country_code, device_code, category_id from  rank_unified ) as unified
group by  category_id,  country_code,  device_code );


-- group by unified data
WITH unified_group_data_store AS (
select count(app_id) as unified_count_app_id, count(free_app_download) as unified_count_free_app_download, count(paid_app_download) as unified_count_paid_app_download, count(revenue) as unified_count_revenue,
  country_code as unified_country_code, device_code as unified_device_code from ( select distinct  app_id, free_app_download, paid_app_download, revenue, country_code, device_code from  store_unified ) as unified
group by   country_code,  device_code );




-- compare raw vs unified data
WITH compared_data_rank AS (
    SELECT * from country_category_mapping_rank_raw left join unified_group_data on unified_group_data.unified_country_code==country_category_mapping_rank_raw.country_code and unified_group_data.unified_category_id==country_category_mapping_rank_raw.category_id and unified_group_data.unified_device_code==country_category_mapping_rank_raw.device_code
);

WITH miss_data_rank AS (
select * from compared_data_rank where unified_count_paid_app_download!=paid_app_download or unified_count_free_app_download != free_app_download  or unified_count_revenue != revenue or unified_count_app_id is null
)



-- compare raw vs unified data store
WITH compared_store_data AS (
    SELECT * from country_mapping_store_raw left join unified_group_data_store on unified_group_data_store.unified_country_code==country_mapping_store_raw.country_code and unified_group_data_store.unified_device_code==country_mapping_store_raw.device_code
);


WITH miss_data_store AS (
select * from compared_store_data where free_app_download!=unified_count_free_app_download or paid_app_download!=unified_count_paid_app_download or revenue!=unified_count_revenue or unified_count_app_id is null
)


"""

dates = ["2010-07-04","2010-07-05","2010-07-06","2010-07-07","2010-07-08","2010-07-09","2010-07-10"]

d1 = spark.read.csv("s3://b2c-prod-dca-store-estimates/store_back/dimension/IOS_COUNTRY_MAPPING",
                    sep="\t").withColumnRenamed("_c0", "store_id").withColumnRenamed("_c1",
                                                                                     "country_code").withColumn(
    "market_code", F.lit("ios"))
d1 = spark.createDataFrame([(0, 'WW', 'Worldwide', 'ios')],
                           schema=["store_id", "country_code", "_c2", "market_code"]).union(d1)

d2 = spark.read.csv("s3://b2c-prod-dca-store-estimates/store_back/dimension/ANDROID_COUNTRY_MAPPING",
                    sep="\t").withColumnRenamed("_c0", "store_id").withColumnRenamed("_c1",
                                                                                     "country_code").withColumn(
    "market_code", F.lit("android"))

country_code_df = d1.union(d2).where("country_code is not null").cache()
country_code_df = country_code_df.withColumnRenamed("store_id", "country_code_store_id")
print 'country mapping table'
country_code_df.show(2)
country_code_df.createOrReplaceTempView("country_code_mapping")

category_mapping_table = spark.read.parquet(
    "s3://b2c-prod-data-pipeline-qa/aa.store/store_cateogry_mapping")
category_mapping_table.createOrReplaceTempView("category_mapping_deminsion_service")

namespace = "aa.store.market-size.v1"
for test_date in dates:
    print test_date
    csv_schema = T.StructType(
        [
            T.StructField("store_id", T.IntegerType(), True),
            T.StructField("date", T.StringType(), True),
            T.StructField("platform_id", T.IntegerType(), True),
            T.StructField("vertical", T.IntegerType(), True),
            T.StructField("feed", T.IntegerType(), True),
            T.StructField("id", T.LongType(), True),
            T.StructField("est", T.IntegerType(), True),
            T.StructField("category_id", T.IntegerType(), True),
            T.StructField("rank", T.IntegerType(), True)
        ]
    )

    raw2 = spark.read.option("basePath",
                             "s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/").schema(
        csv_schema).csv(
        "s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/{}/ios/sbe_est_app/*/".format(test_date),
        sep="\t").withColumn("platform", F.lit("ios")).cache()
    raw2.createOrReplaceTempView("rank_raw")

    ingest_msg = {
        "namespace": "aa.store.market-size.v1",
        "job_type": "routine",
        "options": {},
        "source": [
            {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "rank_unified",
                "path": [
                    "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/date={}".format(
                        test_date)],
            }, {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "store_unified",
                "path": [
                    "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}".format(
                        test_date)],
            }
        ]
    }
    run(spark, ingest_msg, sql_text)
    result = spark.sql("select * from miss_data_rank").show()
    # result.write.format("delta").save("s3://b2c-prod-data-pipeline-qa/aa.store/result_rank_data_v1_{}/".format(start),
    #                                   mode="append",
    #                                   partitionBy=["date"])


    result_store = spark.sql("select * from miss_data_store").show()
    # result_store.write.format("delta").save("s3://b2c-prod-data-pipeline-qa/aa.store/result_store_data_v1_{}/".format(start),
    #                                   mode="append",
    #                                   partitionBy=["date"])
    eject_all_caches(spark)


In [0]:

# |2013-06-03|      143473|          MY| ios-tablet|     100000|             7059|             6358|   4140|               14530|                           6977|                           6357|                 4137|                  MY|         ios-tablet|             100000|

from pyspark.sql import types as T
from pyspark.sql import functions as F


test_date='2013-06-03'
csv_schema = T.StructType(
    [
        T.StructField("store_id", T.IntegerType(), True),
        T.StructField("date", T.StringType(), True),
        T.StructField("platform_id", T.IntegerType(), True),
        T.StructField("vertical", T.IntegerType(), True),
        T.StructField("feed", T.IntegerType(), True),
        T.StructField("id", T.LongType(), True),
        T.StructField("est", T.IntegerType(), True),
        T.StructField("category_id", T.IntegerType(), True),
        T.StructField("rank", T.IntegerType(), True)
    ]
)

raw1 = spark.read.option("basePath",
                         "s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/").schema(
    csv_schema).csv(
    "s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/{}/android/sbe_est_app/*/".format(
        test_date), sep="\t").withColumn("platform", F.lit("android")).cache()
raw2 = spark.read.option("basePath",
                         "s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/").schema(
    csv_schema).csv(
    "s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/{}/ios/sbe_est_app/*/".format(test_date),
    sep="\t").withColumn("platform", F.lit("ios")).cache()

df_raw = raw1.union(raw2).cache()
df_raw.createOrReplaceTempView("rank_raw")


print spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/date=2013-06-03").filter("country_code='MY' and device_code='ios-tablet' and category_id=100000 and free_app_download is not null").count()
spark.sql("select count(*) from rank_raw where store_id='143473' and category_id=36 and feed=101").show()




df_u = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/date=2013-06-03").filter("country_code='MY' and device_code='ios-tablet' and category_id=100000 and free_app_download is not null").select("app_id")
df_r = spark.sql("select id as app_id from rank_raw where store_id='143473' and category_id=36 and feed=101")

df_u.createOrReplaceTempView("d1")
df_r.createOrReplaceTempView("d2")

a1 = spark.sql("select * from d2 except all select * from d1").collect()
spark.sql("select * from d1 except all select * from d2").show()



# |      date|raw_store_id|country_code|device_code|category_id|free_app_download|paid_app_download|revenue|unified_count_app_id|unified_count_free_app_download|unified_count_paid_app_download|unified_count_revenue|unified_country_code|unified_device_code|unified_category_id|
# +----------+------------+------------+-----------+-----------+-----------------+-----------------+-------+--------------------+-------------------------------+-------------------------------+---------------------+--------------------+-------------------+-------------------+
# |2013-06-03|      143473|          MY| ios-tablet|     100000|             7059|             6358|   4140|               14530|                           6977|                           6357|                 4137|                  MY|         ios-tablet|             100000|
# +----------+------------+------------+-----------+-----------+-----------------+-----------------+-------+--------------------+-------------------------------+-------------------------------+---------------------+--------------------+-------------------+-------------------+





In [0]:
%%sh

aws s3 ls s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2013-06-03/ios/sbe_est_app/143473/


In [0]:

print 'raw'
spark.sql("select * from rank_raw where store_id='143473' and category_id=36 and feed=101 and id=638447236").show()

print 'unified'
spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/date=2013-06-03").filter("country_code='MY' and device_code='ios-tablet' and category_id=100000 and free_app_download is not null and app_id=638447236").show()



In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
select  app_id,feed_id,store_id,estimate,category_id ,device_id from plproxy.execute_select_nestloop(\$proxy\$ 
select  app_id,feed_id,store_id,estimate,category_id ,device_id
    from aa.app_store_daily_estimate_1
    where 
      date = '2013-06-03' and store_id=143473 and feed_id = 101 and category_id=36 and app_id=638447236 order by estimate  asc limit 5 \$proxy\$) tbl 
      (app_id BIGINT,feed_id SMALLINT,store_id INT, estimate INT, category_id INT, device_id SMALLINT ) order by estimate  asc limit 5 ;

EOF

In [0]:

raw2 = spark.read.option("basePath",
                         "s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/").schema(
    csv_schema).csv(
    "s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/{}/ios/sbe_est_app/143473".format("2013-06-03"),
    sep="\t").withColumn("platform", F.lit("ios")).cache()

raw2.createOrReplaceTempView("raw2")
spark.sql("select * from raw2 where store_id='143473' and category_id=36 and feed=101 and id in (503834631, 424162537, 458568213, 368358948, 501709193, 629752226, 538679511, 368360223, 304006512, 542557212, 450407440, 599158061, 324715238, 359737307, 638447236, 397425651, 458302072, 583436811, 335485983, 359354972, 284815942, 592784743, 640972086, 349166314, 600476372, 502998318, 397730127, 327761807, 303692704, 609434838, 469623568, 392583941, 600907460, 364740856, 397467588, 379450383, 291877741, 378478530, 513765092, 291179703, 438477986, 309627313, 546249827, 341922306, 384297569, 398833336, 631885241, 452999495, 401424333, 364195592, 437203244, 367278030, 282935706, 573814137, 404750273, 410525831, 364879902, 584986228, 367163867, 428231977, 429515063, 568032967, 288499125, 291070079, 591633237, 414706506, 519401597, 349554263, 424494669, 529856977, 494875526, 546175939, 519050630, 433349450, 493794484, 530262913, 472182350, 326079517, 349039721, 490225166, 388358640, 454812087) order by id , rank ").show(500)


In [0]:
%%sh
